Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: C:\dev\reinforcement_learning\batch\aml_config\config.json


In case if you do not have, any you can either create it on Azure portal or programmatically:

In [ ]:
# from azureml.core import Workspace
# ws = Workspace.create(name = "cheng_offline_dashboard",
#                       subscription_id = "9a2d7383-3c7d-492c-94fc-ba65be408672",
#                       resource_group = "cps-dev-cheng",
#                       create_resource_group = False,
#                       location = 'eastus'
#                      )
# ws.write_config()
# ws.get_details()

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
ws.datastores

In [2]:
import application
from application import context

ctx = context.Context(
    accountName = 'chengmystorage',
    accountKey = 'OOvXDYUpXqfd9kyipcGyThuVQqh+avvHl36hRcBjeA53FBBtLkx0L0dRJhqpMJV4dXVfYGOWngH9qols20rzPQ==',
    appId = 'large',
    appFolder = 'folder'
)

Now you can create batch training pipeline for vw batch training:

Created pipeline can be either submitted for execution:

Or published to be available via REST endpoint:

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [3]:
from pipelines import extract_pipeline
extractPipeline = extract_pipeline.create_pipeline(ws, ctx, parallel_jobs = 100)

Data store for the app context {chengmystoragelarge} is found
Found existing cluster: extractor
Cache step is successfully created
Found existing cluster: python
Best command selection step is created successfully
Found existing cluster: vw-predict
Vw predict step is successfully created
extractPipeline is succesfully created.
Step Predict is ready to be created [f11526b2]
extractPipeline is succesfully validated.


In [4]:
from azureml.core import Experiment  
regen_outputs = False
extractExperiment = Experiment(ws, 'test_large_file').submit(
    extractPipeline, 
    pipeline_params = {
        "start_date":'10/21/2018',
        "end_date":'10/22/2018'
    }, 
    regenerate_outputs = regen_outputs
)
print("Extract Pipeline is submitted for execution")
from azureml.widgets import RunDetails
RunDetails(extractExperiment).show()

Created step Cache [a1f6b79e][8ad35834-6c41-4249-8475-c1f5d046f3d3], (This step is eligible to reuse a previous run's output)
Created step Predict [f11526b2][6d99086d-20e9-44d8-bb80-561a86a5ae01], (This step will run and generate new outputs)
Created step Prepare command [4e00bc0d][6e3fe8b9-6911-43c1-b177-37a7e7ca0c8b], (This step is eligible to reuse a previous run's output)
Using data reference Application_logs for StepId [59431f93][5b4385df-17be-405c-87e2-03be22dccecc], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: 1bdac811-932b-4ff1-b67c-90f4abf85f86
Extract Pipeline is submitted for execution


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …